### Data Cleaning Steps
  
Two datasets will be created; one with all variables encoded to numeric values and missing values imputed for models such as regression, KNN, SVM. Another with minimal data preprocessing for the boosting algorithms. the same processing must be done on the test dataset.  

#### Dataset 1
1) Remove variables/columns that have more than 30% of responses missing

#### Dataset 2

1) Categorize any missing response from (NA, "", ., .a, .b, .c, .d) to -1  
2) Remove variables/columns that have more than 30% of responses missing  
4) Encode country and language responses  
5) Clean data types of variables/columns to corresponding str/float/int64  



## Dataset 1

In [1]:
# Import packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

# Set options
pd.options.display.max_rows = 999

In [2]:
# Import data
train = pd.read_csv("../01-data/train.csv", low_memory = False)
test = pd.read_csv("../01-data/test.csv", low_memory = False)

# Fill missing responses with "." so that they can be counted and categorized as missing later on
train_no_blanks = train.fillna('.')
test_no_blanks = test.fillna('.')

Training set:

In [3]:
# Treating missing 
frequency_train_data = pd.DataFrame()

for i in list(train_no_blanks)[1:]:
    grouped_data = train_no_blanks.groupby(i)["id"].count()
    num_unique_answers = grouped_data.size
    temp_dict = {"Question": [i] * num_unique_answers,
                 "Response": list(grouped_data.index),
                 "Frequency": list(grouped_data)}
    frequency_train_data = frequency_train_data.append(pd.DataFrame(temp_dict))

frequency_train_data.reset_index(inplace=True, drop=True)
temp = frequency_train_data.groupby(["Question","Response"]).agg({"Frequency":"sum"})
temp = temp.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))
temp.reset_index(inplace=True)
temp = temp.rename(columns = {"Frequency":"Relative Frequency (%)"})
frequency_train_data = frequency_train_data.merge(temp, left_on = ["Question","Response"], right_on = ["Question","Response"], how = "left")

# If response contains a "." (., .a, .b, etc) then categorize as missing a response
frequency_train_data['Response Missing'] = pd.np.where(frequency_train_data.Response.str.find(".") > -1, 1, 0)

# Calculate # missing for each question
missing_value_df = frequency_train_data.groupby(["Question","Response","Response Missing"]).agg({"Frequency":"sum"})
missing_value_df = missing_value_df.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))
missing_value_df = missing_value_df.groupby(["Question","Response Missing"]).agg({"Frequency":"sum"})
missing_value_df = missing_value_df[missing_value_df.index.get_level_values("Response Missing")==1]
missing_value_df = missing_value_df[missing_value_df.index.get_level_values("Question")!="satisfied"]
missing_value_df.reset_index(inplace=True)
missing_value_df = missing_value_df.rename(columns = {"Frequency":"Percentage Missing"}).drop("Response Missing", axis=1)

# Rank missing and drop
percent = 30
cols_missing = missing_value_df[missing_value_df["Percentage Missing"] > percent]
n_cols_missing = len(missing_value_df[missing_value_df["Percentage Missing"] > percent])

drop_missing = cols_missing.Question.to_list()
train_v2 = train_no_blanks.drop(columns = drop_missing)

In [4]:
# Write data
train_v2.to_csv("train_processed_1.csv")

Test set:

In [6]:
# Treating missing 
frequency_test_data = pd.DataFrame()

for i in list(test_no_blanks)[1:]:
    grouped_data = test_no_blanks.groupby(i)["id"].count()
    num_unique_answers = grouped_data.size
    temp_dict = {"Question": [i] * num_unique_answers,
                 "Response": list(grouped_data.index),
                 "Frequency": list(grouped_data)}
    frequency_test_data = frequency_test_data.append(pd.DataFrame(temp_dict))

frequency_test_data.reset_index(inplace=True, drop=True)
temp = frequency_test_data.groupby(["Question","Response"]).agg({"Frequency":"sum"})
temp = temp.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))
temp.reset_index(inplace=True)
temp = temp.rename(columns = {"Frequency":"Relative Frequency (%)"})
frequency_test_data = frequency_test_data.merge(temp, left_on = ["Question","Response"], right_on = ["Question","Response"], how = "left")

# If response contains a "." (., .a, .b, etc) then categorize as missing a response
frequency_test_data['Response Missing'] = pd.np.where(frequency_test_data.Response.str.find(".") > -1, 1, 0)

# Calculate # missing for each question
missing_value_df = frequency_test_data.groupby(["Question","Response","Response Missing"]).agg({"Frequency":"sum"})
missing_value_df = missing_value_df.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))
missing_value_df = missing_value_df.groupby(["Question","Response Missing"]).agg({"Frequency":"sum"})
missing_value_df = missing_value_df[missing_value_df.index.get_level_values("Response Missing")==1]
missing_value_df = missing_value_df[missing_value_df.index.get_level_values("Question")!="satisfied"]
missing_value_df.reset_index(inplace=True)
missing_value_df = missing_value_df.rename(columns = {"Frequency":"Percentage Missing"}).drop("Response Missing", axis=1)

# Rank missing and drop
percent = 30
cols_missing = missing_value_df[missing_value_df["Percentage Missing"] > percent]
n_cols_missing = len(missing_value_df[missing_value_df["Percentage Missing"] > percent])

drop_missing = cols_missing.Question.to_list()
test_v2 = test_no_blanks.drop(columns = drop_missing)

In [7]:
# Write data
test_v2.to_csv("test_processed_1.csv")

## Dataset 2

Training set:

In [26]:
# Group all missing responses and code them as -1
train_v2 = train_v2.replace([".", ".a", ".b", ".c", ".d"], [-1, -1, -1, -1, -1])

# Dealing with non-numeric
cat_cols = ["v17", "v20", "v25", "v78", "v154", "v155", "v161", "cntry"]

# change variable type to str
train_v2[cat_cols] = train_v2[cat_cols].astype(str)

# Copy dataset
train_v3 = train_v2.copy()

# Label encoding the country variables
label_encoder = preprocessing.LabelEncoder()

for col in cat_cols:
    train_v3[col] = label_encoder.fit_transform(train_v3[col])
    
# Convert all other variables to int64
num_cols = train_v3.loc[:, ~train_v3.columns.isin(cat_cols)].columns.tolist()
train_v3[num_cols] = train_v3[num_cols].astype("int64")

Checks:

In [38]:
display(any(train_v3.dtypes == object))
display(any(train_v3.dtypes == str))

False

False

In [39]:
# Write data
train_v3.to_csv("train_processed_2.csv")

Test set:

In [40]:
# Group all missing responses and code them as -1
test_v2 = test_v2.replace([".", ".a", ".b", ".c", ".d"], [-1, -1, -1, -1, -1])

# Dealing with non-numeric
cat_cols = ["v17", "v20", "v25", "v78", "v154", "v155", "v161", "cntry"]

# change variable type to str
test_v2[cat_cols] = test_v2[cat_cols].astype(str)

# Copy dataset
test_v3 = test_v2.copy()

# Label encoding the country variables
label_encoder = preprocessing.LabelEncoder()

for col in cat_cols:
    test_v3[col] = label_encoder.fit_transform(test_v3[col])
    
# Convert all other variables to int64
num_cols = test_v3.loc[:, ~test_v3.columns.isin(cat_cols)].columns.tolist()
test_v3[num_cols] = test_v3[num_cols].astype("int64")

Checks:

In [41]:
display(any(test_v3.dtypes == object))
display(any(test_v3.dtypes == str))

False

False

In [42]:
# Write data
test_v3.to_csv("test_processed_2.csv")